In [2]:
    from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
    from torch.utils.data import Dataset
    import torch

    # Nom du modèle et des fichiers
    MODEL_NAME = "gpt2"
    DATA_PATH = "data_team.txt"  # Fichier avec les questions et équipes
    OUTPUT_DIR = "fine_tuned_gpt2-3"

    # Charger le tokenizer et le modèle pré-entraîné
    tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
    model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

    # Ajouter des tokens spéciaux
    tokenizer.add_special_tokens({"pad_token": "[PAD]"})  # Token de padding
    model.resize_token_embeddings(len(tokenizer))

    # Définition du Dataset
    class PokemonTeamDataset(Dataset):
        def __init__(self, file_path, tokenizer, block_size):
            with open(file_path, "r", encoding="utf-8") as file:
                data = file.read()

            # Séparer les exemples avec "---"
            examples = data.split("---")

            # Tokeniser chaque exemple (ajoute implicitement "Donne-moi une équipe :" au début)
            self.examples = [
                tokenizer(f"{example.strip()}", max_length=block_size, truncation=True, padding="max_length")["input_ids"]
                for example in examples if example.strip()
            ]

        def __len__(self):
            return len(self.examples)

        def __getitem__(self, idx):
            return torch.tensor(self.examples[idx])

    # Paramètres
    BLOCK_SIZE = 128  # Taille des tokens
    train_dataset = PokemonTeamDataset(DATA_PATH, tokenizer, BLOCK_SIZE)

    # Collator pour gérer le padding
    from transformers import DataCollatorForLanguageModeling

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,  # On entraîne en mode génération, pas en masquage
    )

    # Configuration de l'entraînement
    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        num_train_epochs=10,  # Augmenter le nombre d'époques pour un meilleur apprentissage
        per_device_train_batch_size=2,
        save_steps=500,
        save_total_limit=2,
        logging_dir=f"{OUTPUT_DIR}/logs",
        logging_steps=50,
        overwrite_output_dir=True,
        evaluation_strategy="no",
    )

    # Instancier Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )

    # 🚀 Entraînement
    trainer.train()

    # Sauvegarde du modèle et du tokenizer
    model.save_pretrained(OUTPUT_DIR)
    tokenizer.save_pretrained(OUTPUT_DIR)

    print(f"✅ Modèle fine-tuné sauvegardé dans {OUTPUT_DIR}")


c:\Users\lolol\anaconda3\envs\llm\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/4340 [00:00<?, ?it/s]

{'loss': 2.793, 'grad_norm': 16.11667251586914, 'learning_rate': 4.9423963133640554e-05, 'epoch': 0.12}
{'loss': 1.645, 'grad_norm': 18.171791076660156, 'learning_rate': 4.8847926267281106e-05, 'epoch': 0.23}
{'loss': 1.4873, 'grad_norm': 13.069515228271484, 'learning_rate': 4.827188940092166e-05, 'epoch': 0.35}
{'loss': 1.4074, 'grad_norm': 13.665339469909668, 'learning_rate': 4.7695852534562216e-05, 'epoch': 0.46}
{'loss': 1.2604, 'grad_norm': 11.04793643951416, 'learning_rate': 4.711981566820277e-05, 'epoch': 0.58}
{'loss': 1.2453, 'grad_norm': 14.253957748413086, 'learning_rate': 4.654377880184332e-05, 'epoch': 0.69}
{'loss': 1.234, 'grad_norm': 11.726035118103027, 'learning_rate': 4.596774193548387e-05, 'epoch': 0.81}
{'loss': 1.1952, 'grad_norm': 11.10894775390625, 'learning_rate': 4.539170506912442e-05, 'epoch': 0.92}
{'loss': 1.2199, 'grad_norm': 13.73951244354248, 'learning_rate': 4.4815668202764974e-05, 'epoch': 1.04}
{'loss': 1.109, 'grad_norm': 11.516401290893555, 'learning